In [1]:
from datasets import load_dataset

In [2]:
egset = load_dataset("abairxml.py", data_dir=f"/home/jim/tmp/cmg_ga_mu/CMGCorpusBeag")

Using custom data configuration default-be79e07535743a6e
Reusing dataset utterance_xml_dataset (/home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-be79e07535743a6e/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b)


In [3]:
egset

DatasetDict({
    train: Dataset({
        features: ['file_id', 'words', 'phonemes', 'audio', 'dialect', 'language', 'speaker_id', 'audio_set'],
        num_rows: 1453
    })
})

In [4]:
from pathlib import Path
def check_paths(batch):
    audio_path = Path(batch["audio"])
    return audio_path.is_file()

egset = egset.filter(check_paths)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-be79e07535743a6e/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-b0de8317eeb157fe.arrow


In [6]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["audio"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["words"]
    return batch

egset = egset.map(speech_file_to_array_fn, remove_columns=egset['train'].column_names)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-be79e07535743a6e/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-ae73b888f9fe20f5.arrow


In [7]:
egset

DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 1453
    })
})

In [9]:
import librosa
import numpy as np

In [10]:
def resample(batch):
    batch["duration"] = librosa.get_duration(y=batch["speech"], sr=batch["sampling_rate"])

    if batch["sampling_rate"] != 16_000:
        batch["speech"] = librosa.resample(np.asarray(batch["speech"]), batch["sampling_rate"], 16_000)
        batch["sampling_rate"] = 16_000

    return batch
egset = egset.map(resample, num_proc=4)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-be79e07535743a6e/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-5f5e02953a0c6c6b.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-be79e07535743a6e/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-8587b9046b2a82a2.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-be79e07535743a6e/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-376ba06e76353015.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-be79e07535743a6e/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-7d11fe2d6553624f.arrow


In [16]:
egset = egset.sort('duration')

Loading cached sorted indices for dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-be79e07535743a6e/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-53da9334ce2f906e.arrow


In [18]:
egset['train'][0]['duration']

1.5309375

In [19]:
egset['train'][-1]['duration']

7.406125